# Mech Tools Evaluation

### Create the dataset

In [ ]:
!docker build -t etl:0.0.1 . && docker run etl:0.0.1

### Load the dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option("display.precision", 2)
pd.set_option('display.max_columns', None)

In [2]:
dataset = pd.read_csv("dataset.csv")
str_cols = ("id", "currentAnswer", "title", "request_id", "prompt", "tool", "nonce", "vote")
for col in str_cols:
    dataset[col] = dataset[col].astype("string")
dataset.head()

,id,currentAnswer,title,request_id,request_block,prompt,tool,nonce,deliver_block,p_yes,p_no,confidence,info_utility,vote,win_probability
0,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,1429730407779530824523722231071959771311408049...,29544655,"With the given question ""Will the Hisense U8K ...",prediction-online,c6366b3f-eff5-4533-8dd9-d653b281b29d,29577379,0.60,0.40,0.8,0.5,Yes,0.60
1,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,1695055931594747475916883029584567955775422500...,29545478,"With the given question ""Will the Hisense U8K ...",prediction-online,1eed33a5-a3f0-41c4-beae-23e9022ffe22,29576660,0.60,0.40,0.8,0.7,Yes,0.60
2,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,5972945302788386668720465960403202339977906500...,29546230,"With the given question ""Will the Hisense U8K ...",prediction-online,dd376ef9-eb2c-4d9f-8a5a-cf9ae8deb0b3,29576574,0.60,0.40,0.8,0.7,Yes,0.60
3,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,1043402953919313937539182160739114840263108832...,29546982,"With the given question ""Will the Hisense U8K ...",prediction-online,91096f15-5e3b-4bf1-8178-f17f1efcf639,29576448,0.70,0.30,0.8,0.6,Yes,0.70
4,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,9433232780766388309643050548812272093999565778...,29547744,"With the given question ""Will the Hisense U8K ...",prediction-online,92321968-7888-4877-b33f-22fa4755fbc2,29576351,0.65,0.35,0.9,0.8,Yes,0.65


In [3]:
dataset.shape

(17616, 15)

In [4]:
dataset.describe()

,request_block,deliver_block,p_yes,p_no,confidence,info_utility,win_probability
count,1.76e+04,1.76e+04,17616.00,17616.00,17616.00,17616.00,17616.00
mean,3.04e+07,3.04e+07,0.48,0.52,0.78,0.57,0.68
std,5.55e+05,5.54e+05,0.20,0.20,0.08,0.18,0.09
min,2.90e+07,2.90e+07,0.00,0.00,0.50,0.00,0.53
25%,2.99e+07,2.99e+07,0.30,0.40,0.80,0.50,0.60
50%,3.05e+07,3.05e+07,0.60,0.40,0.80,0.60,0.70
75%,3.10e+07,3.10e+07,0.60,0.70,0.80,0.60,0.75
max,3.12e+07,3.12e+07,1.00,1.00,1.00,1.00,1.00


In [5]:
dataset["vote"].describe()

count     17616
unique        2
top         Yes
freq      10238
Name: vote, dtype: object

### Normalize confidences

In [6]:
dataset["confidence"].unique()

array([0.8 , 0.9 , 0.7 , 0.5 , 0.6 , 0.85, 0.75, 1.  , 0.99, 0.95])

In [7]:
dataset.loc[(dataset["confidence"] >= 0.9) & (dataset["confidence"] < 1), "confidence"] = 0.9
dataset.loc[dataset["confidence"] == 0.85, "confidence"] = 0.8
dataset.loc[dataset["confidence"] == 0.75, "confidence"] = 0.7
dataset["confidence"].unique()

array([0.8, 0.9, 0.7, 0.5, 0.6, 1. ])

### Check the percentage of wins vs confidence for all the tools

In [8]:
def accuracy(data):
    correct_answers_mask = data["currentAnswer"] == data["vote"]
    n_answers = correct_answers_mask.count()
    n_answers_success = correct_answers_mask.sum()

    if n_answers == 0:
        accuracy = None
    elif n_answers_success == 0:
        accuracy = 0
    else:
        accuracy = n_answers_success/n_answers * 100

    return pd.Series({"n_correct": n_answers_success, "n_pred": n_answers, "accuracy": accuracy})

In [9]:
def acc_per_tool(group, col: str, conf: float):
    return group.apply(lambda x: accuracy(x[x[col] == conf]))

In [10]:
def gen_stats(group, col: str):
    stats = {f"{col}_{prob}" : acc_per_tool(group, col, prob) for prob in sorted(dataset[col].unique())}
    stats["total"] = tools_group.apply(accuracy)
    return pd.concat(stats.values(), axis=1, keys=stats.keys())

In [11]:
tools_group = dataset.groupby("tool")
tools_stats_per_conf = gen_stats(tools_group, "confidence")
display(tools_stats_per_conf)

confidence_0.5                  \
                                                n_correct n_pred accuracy   
tool                                                                        
claude-prediction-offline                           297.0  434.0    68.43   
claude-prediction-online                             63.0   84.0    75.00   
prediction-offline                                    0.0    0.0      NaN   
prediction-offline-sme                                0.0    0.0      NaN   
prediction-online                                     0.0    0.0      NaN   
prediction-online-sme                                 0.0    0.0      NaN   
prediction-online-summarized-info                     0.0    0.0      NaN   
prediction-sentence-embedding-bold                    0.0    0.0      NaN   
prediction-sentence-embedding-conservative            0.0    0.0      NaN   

                                           confidence_0.6                  \
                                                n_correct n_pred accuracy   
tool                                                                        
claude-prediction-offline                            19.0   43.0    44.19   
claude-prediction-online                             97.0  129.0    75.19   
prediction-offline                                    9.0    9.0   100.00   
prediction-offline-sme                               32.0   40.0    80.00   
prediction-online                                    52.0   61.0    85.25   
prediction-online-sme                                79.0  118.0    66.95   
prediction-online-summarized-info                     0.0    1.0     0.00   
prediction-sentence-embedding-bold                    0.0    0.0      NaN   
prediction-sentence-embedding-conservative            0.0    0.0      NaN   

                                           confidence_0.7                   \
                                                n_correct  n_pred accuracy   
tool                                                                         
claude-prediction-offline                            81.0   191.0    42.41   
claude-prediction-online                            189.0   332.0    56.93   
prediction-offline                                   73.0    94.0    77.66   
prediction-offline-sme                              133.0   160.0    83.12   
prediction-online                                   862.0  1125.0    76.62   
prediction-online-sme                               608.0   919.0    66.16   
prediction-online-summarized-info                     0.0     3.0     0.00   
prediction-sentence-embedding-bold                    0.0     1.0     0.00   
prediction-sentence-embedding-conservative            0.0     0.0      NaN   

                                           confidence_0.8                   \
                                                n_correct  n_pred accuracy   
tool                                                                         
claude-prediction-offline                            15.0    28.0    53.57   
claude-prediction-online                             21.0    47.0    44.68   
prediction-offline                                  220.0   559.0    39.36   
prediction-offline-sme                              281.0   627.0    44.82   
prediction-online                                  3587.0  8010.0    44.78   
prediction-online-sme                              1465.0  2926.0    50.07   
prediction-online-summarized-info                     8.0    13.0    61.54   
prediction-sentence-embedding-bold                    0.0     1.0     0.00   
prediction-sentence-embedding-conservative            0.0     0.0      NaN   

                                           confidence_0.9                   \
                                                n_correct  n_pred accuracy   
tool                                                                         
claude-prediction-offline                            28.0    34.0    82.35   
claude-

### Check the percentage of wins vs probability for all the tools

In [12]:
tools_stats_per_prob = gen_stats(tools_group, "win_probability")
tools_stats_per_prob

win_probability_0.525         \
                                                       n_correct n_pred   
tool                                                                      
claude-prediction-offline                                    0.0    0.0   
claude-prediction-online                                     0.0    0.0   
prediction-offline                                           0.0    0.0   
prediction-offline-sme                                       0.0    0.0   
prediction-online                                            0.0    0.0   
prediction-online-sme                                        0.0    1.0   
prediction-online-summarized-info                            0.0    0.0   
prediction-sentence-embedding-bold                           0.0    0.0   
prediction-sentence-embedding-conservative                   0.0    0.0   

                                                    win_probability_0.55  \
                                           accuracy            n_correct   
tool                                                                       
claude-prediction-offline                       NaN                  0.0   
claude-prediction-online                        NaN                  1.0   
prediction-offline                              NaN                  1.0   
prediction-offline-sme                          NaN                  0.0   
prediction-online                               NaN                 32.0   
prediction-online-sme                           0.0                 11.0   
prediction-online-summarized-info               NaN                  0.0   
prediction-sentence-embedding-bold              NaN                  0.0   
prediction-sentence-embedding-conservative      NaN                  0.0   

                                                            \
                                           n_pred accuracy   
tool                                                         
claude-prediction-offline                     2.0     0.00   
claude-prediction-online                      1.0   100.00   
prediction-offline                            3.0    33.33   
prediction-offline-sme                        1.0     0.00   
prediction-online                            54.0    59.26   
prediction-online-sme                        19.0    57.89   
prediction-online-summarized-info             0.0      NaN   
prediction-sentence-embedding-bold            0.0      NaN   
prediction-sentence-embedding-conservative    0.0      NaN   

                                           win_probability_0.6          \
                                                     n_correct  n_pred   
tool                                                                     
claude-prediction-offline                                140.0   303.0   
claude-prediction-online                                 177.0   324.0   
prediction-offline                                       105.0   310.0   
prediction-offline-sme                                   115.0   336.0   
prediction-online                                       1695.0  4546.0   
prediction-online-sme                                    761.0  1655.0   
prediction-online-summarized-info                          7.0     9.0   
prediction-sentence-embedding-bold                         0.0     1.0   
prediction-sentence-embedding-conservative                 0.0     0.0   

                                                    win_probability_0.65  \
                                           accuracy            n_correct   
tool                                                                       
claude-prediction-offline                     46.20                  2.0   
claude-prediction-online                      54.63                  9.0   
prediction-offline                            33.87                  5.0   
prediction-offline-sme                        34.23                  5.0   
prediction-online                             37.29            

### List the well performing tools

In [13]:
# we are using 55% as the performance threshold and 100 as the samples threshold
perf_threshold = 55
samples_threshold = 100

In [14]:
well_performing = tools_stats_per_conf.loc[:, ("total", "accuracy")] > perf_threshold
enough_samples = tools_stats_per_conf.loc[:, ("total", "n_pred")] > samples_threshold
tools_stats_per_conf.loc[well_performing & enough_samples, [("total", "accuracy"), ("total", "n_pred")]]

total       
                          accuracy n_pred
tool                                     
claude-prediction-offline    61.18  765.0
claude-prediction-online     62.25  596.0